In [7]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import random_split
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import os

MODEL_NUMBER = 7

    
# seed so random stuff isnt random
SEED = 100

# transforms with imagenet normalization
transform = transforms.Compose([
    transforms.RandomResizedCrop(64, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),

])

dataset = torchvision.datasets.EuroSAT(root='./data', download=True, transform=transform)

# split into train, test
# 80% train, 20% test 
train_ratio = 0.8
test_ratio = 0.2


total_size = len(dataset)
train_size = int(total_size * train_ratio)
test_size = total_size - train_size


train_dataset, test_dataset = random_split(
    dataset, [train_size, test_size],
    generator=torch.Generator().manual_seed(SEED)
)

print(f"train: {len(train_dataset)}")
print(f"test size: {len(test_dataset)}")

train: 21600
test size: 5400


In [8]:
import torchvision.models as models

# convnext_large model pretrained on imagenet
model = models.convnext_large(weights=models.ConvNeXt_Large_Weights.IMAGENET1K_V1)


# classifier head with 10 outputs
num_ftrs = model.classifier[2].in_features
model.classifier[2] = torch.nn.Linear(num_ftrs, 10)

print(f"Model: ConvNeXt-Large (ImageNet pretrained)")
print(f"Total parameters: {sum(p.numel() for p in model.parameters())}")


Model: ConvNeXt-Large (ImageNet pretrained)
Total parameters: 196245706


In [9]:

# dataloaders
train_loader = torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=128, 
    shuffle=True,
    pin_memory=True, # for use on gpu
    num_workers=4 # arbitrarily chosen. some sources recommend 4 per gpu
)


test_loader = torch.utils.data.DataLoader(
    test_dataset, 
    batch_size=128, 
    shuffle=False,
    pin_memory=True,
    num_workers=4
)

In [10]:

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

device = "cuda" if torch.cuda.is_available() else "cpu"

if device == "cuda":
    model.to(device, non_blocking=True) # according to pytorch docs we can do this if memory is pinned
else:  
    model.to(device)
    


In [11]:
def train_one_epoch():
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    avg_loss = total_loss / len(train_loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy

def test_one_epoch():
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    avg_loss = total_loss / len(test_loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy

In [ ]:
# tensorboard setup
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/train_{}_{}'.format(MODEL_NUMBER, timestamp))

checkpoint_dir = f'checkpoints/train_{MODEL_NUMBER}_{timestamp}'
os.makedirs(checkpoint_dir, exist_ok=True)

EPOCHS = 1000

for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    


    train_loss, train_acc = train_one_epoch()
    
    test_loss, test_acc = test_one_epoch()
    
    print(f'train loss: {train_loss:.4f} | train acc: {train_acc:.2f}%')
    print(f'test loss:  {test_loss:.4f} | test acc:  {test_acc:.2f}%')
    
    # log data
    writer.add_scalar('loss/1_train', train_loss, epoch + 1)
    writer.add_scalar('loss/2_test', test_loss, epoch + 1)
    writer.add_scalar('accuracy/1_train', train_acc, epoch + 1)
    writer.add_scalar('accuracy/2_test', test_acc, epoch + 1)
    writer.flush()
    
    # checkpoint every 20 epochs
    if (epoch + 1) % 20 == 0:
        checkpoint_path = os.path.join(checkpoint_dir, f'model_epoch_{epoch + 1}.pt')
        torch.save(model.state_dict(), checkpoint_path)
        print(f'checkpoint saved: {checkpoint_path}')
    
    print()

# save final model
final_checkpoint_path = os.path.join(checkpoint_dir, f'model_final.pt')
torch.save(model.state_dict(), final_checkpoint_path)
print(f'final checkpoint saved: {final_checkpoint_path}')

writer.close()
print('Complete')

Epoch 1/1000
train loss: 0.3360 | train acc: 89.23%
test loss:  0.1593 | test acc:  94.91%

Epoch 2/1000
train loss: 0.1798 | train acc: 93.97%
test loss:  0.1650 | test acc:  94.52%

Epoch 3/1000
train loss: 0.1693 | train acc: 94.25%
test loss:  0.1700 | test acc:  94.61%

Epoch 4/1000
train loss: 0.1445 | train acc: 95.06%
test loss:  0.1863 | test acc:  93.67%

Epoch 5/1000
train loss: 0.1255 | train acc: 95.77%
test loss:  0.1375 | test acc:  94.96%

Epoch 6/1000
train loss: 0.1315 | train acc: 95.59%
test loss:  0.1511 | test acc:  94.85%

Epoch 7/1000
train loss: 0.1152 | train acc: 96.12%
test loss:  0.1335 | test acc:  95.41%

Epoch 8/1000
train loss: 0.1141 | train acc: 96.08%
test loss:  0.1449 | test acc:  95.04%

Epoch 9/1000
train loss: 0.1030 | train acc: 96.46%
test loss:  0.1858 | test acc:  94.04%

Epoch 10/1000
train loss: 0.1094 | train acc: 96.22%
test loss:  0.1363 | test acc:  95.22%

Epoch 11/1000
train loss: 0.1009 | train acc: 96.45%
test loss:  0.1462 | test 

KeyboardInterrupt: 